**Indexes for Image Classes:**
0. Black-grass
1. Charlock
2. Cleavers
3. Common Chickweed
4. Common wheat
5. Fat Hen
6. Loose Silky-bent
7. Maize
8. Scentless Mayweed
9. Shepherds Purse
10. Small-flowered Cranesbill
11. Sugar beet

In [1]:
# Mount google drive to notebook.
from google.colab import drive
drive.mount('/content/drive/')

# To force remounting google drive to notebook.
#drive.mount('/content/drive', force_remount = True)

# Dismount google drive from notebook.
#drive.flush_and_unmount()

# Ensure we are in the correct working directory.
%cd /content/drive/MyDrive/CX4041-Group-Project/
%ls\

Mounted at /content/drive/
/content/drive/MyDrive/CX4041-Group-Project
 Augmented_Test_Images/
 Augmented_Train_Images/
 Checkpoint/
 CSV/
'CX4041 Group Project Report.docx'
 Model-Figures/
 Notebooks/
 Preprocessed_Test_Images/
 Preprocessed_Train_Images/
'[Video Presentation] StoryBoard & Script.gdoc'


In [2]:
# Import the required libraries.
import os
import pandas as Pandas
import numpy as Numpy
from keras.models import load_model
from natsort import natsorted
from PIL import Image

In [3]:
# Load in the pretrained model checkpoint optimal weights and bias.
efficientnet_model = load_model('/content/drive/MyDrive/CX4041-Group-Project/Checkpoint/EfficientNet-Checkpoint')
inceptionresnetv2_model = load_model('/content/drive/MyDrive/CX4041-Group-Project/Checkpoint/InceptionResNetV2-Checkpoint')
xception_model = load_model('/content/drive/MyDrive/CX4041-Group-Project/Checkpoint/Xception-Checkpoint')

print(type(efficientnet_model))
print(type(inceptionresnetv2_model))
print(type(xception_model))

# Define and combine the models.
plant_combined_models = []
plant_combined_models.append(efficientnet_model)
plant_combined_models.append(inceptionresnetv2_model)
plant_combined_models.append(xception_model)

/usr/local/lib/python3.7/dist-packages/keras/layers/core/lambda_layer.py:305: UserWarning: keras.applications.inception_resnet_v2 is not loaded, but a Lambda layer uses it. It may cause errors.
  'function_type')


<class 'keras.engine.functional.Functional'>
<class 'keras.engine.functional.Functional'>
<class 'keras.engine.functional.Functional'>


In [4]:
# Create a dictionary matching prediction index to their classes.
species_dictionary = {
    0: 'Black-grass',
    1: 'Charlock',
    2: 'Cleavers',
    3: 'Common Chickweed',
    4: 'Common wheat',
    5: 'Fat Hen',
    6: 'Loose Silky-bent',
    7: 'Maize',
    8: 'Scentless Mayweed',
    9: 'Shepherds Purse',
    10: 'Small-flowered Cranesbill',
    11: 'Sugar beet'
}

In [5]:
# Define the function to combine the models in an ensemble and make the final prediction.
def ensemble_predictions(members, image):

  # Make a prediction for each model as a list of lists.
  all_predictions = [model.predict(image) for model in members]

  # Convert the list of predicted probabilities into an array.
  all_predictions = Numpy.array(all_predictions)

  # Sum up the predicted probabilities for each model.
  summed_probabilities = Numpy.sum(all_predictions, axis = 0)

  # Predict the class based on the index with the highest probability in the given row.
  result = Numpy.argmax(summed_probabilities, axis = 1)
  #print('Predicted index:', result)

  # Convert the predicted class index from numpy ndarray into a numpy integer.
  result = result[0]
  return result

In [6]:
# Directory where the test images are stored.
test_images_path = '/content/drive/MyDrive/CX4041-Group-Project/Preprocessed_Test_Images/test/'
test_images_list = []
count = 0

# For each item in the directory of test images.
for test_image_name in os.listdir(test_images_path):

  # Check if the item is a file.
  if os.path.isfile(test_images_path + test_image_name):

    # Add the test image name to a list.
    test_images_list.append(test_image_name)
    count += 1

print('Number of test images detected:', count, '\n')

# Sort the images in the list by name. Use the 'natsorted' package to ensure the filenames are sorted properly by number.
test_images_list = natsorted(test_images_list)

Number of test images detected: 794 



In [7]:
# Define the width and height of the input images.
width = 299
height = 299

# Perform resizing of the test images before feeding into the neural network.
# All neural network models used should accept the same image dimensions. 
for image_index in range(len(test_images_list)):

  # Retrieve the current image using the directory and image name.
  current_image = Image.open(test_images_path + test_images_list[image_index])

  # View the properties of the test images before resizing.
  #print(current_image.format, current_image.mode, current_image.size)

  # Resize the images.
  resized_image = current_image.resize((width, height))

  # View the properties of the test images after resizing.
  #print(resized_image.format, resized_image.mode, resized_image.size)

  # Write the resized images back into the list at the same index.
  test_images_list[image_index] = resized_image
  #print('Finished resizing image index:', image_index + 1)

print('Number of test images resized:', image_index + 1)

# For viewing a single test image for debugging.
#test_images_list[0]

Number of test images resized: 794


In [8]:
# List to hold all the predicted classes for the test images in sequence.
test_species_list = []

# Read in each test image one at a time and use the trained model to predict its class.
for image_index in range(len(test_images_list)):

  # Convert the image into float data type for processing by the network.
  current_image = test_images_list[image_index]
  current_image = Numpy.array(current_image, dtype = float)

  # Add an additional dimension at dimension 0 with a value of 1 for processing 1 test image at a time.
  current_image = current_image[Numpy.newaxis, :]

  # Predict an array of probabilities that the image belongs to each class.
  predicted_class_index = ensemble_predictions(plant_combined_models, current_image)

  # Find the species name based on the predicted index and add it to the list of predicted species names.
  species_name = species_dictionary.get(predicted_class_index)
  test_species_list.append(species_name)
  print('Predicted index:', predicted_class_index, ', species:', species_name)

print('Number of test images predicted:', image_index + 1)

Predicted index: 10 , species: Small-flowered Cranesbill
Predicted index: 5 , species: Fat Hen
Predicted index: 11 , species: Sugar beet
Predicted index: 3 , species: Common Chickweed
Predicted index: 11 , species: Sugar beet
Predicted index: 0 , species: Black-grass
Predicted index: 3 , species: Common Chickweed
Predicted index: 5 , species: Fat Hen
Predicted index: 6 , species: Loose Silky-bent
Predicted index: 6 , species: Loose Silky-bent
Predicted index: 5 , species: Fat Hen
Predicted index: 10 , species: Small-flowered Cranesbill
Predicted index: 11 , species: Sugar beet
Predicted index: 8 , species: Scentless Mayweed
Predicted index: 11 , species: Sugar beet
Predicted index: 5 , species: Fat Hen
Predicted index: 8 , species: Scentless Mayweed
Predicted index: 8 , species: Scentless Mayweed
Predicted index: 3 , species: Common Chickweed
Predicted index: 9 , species: Shepherds Purse
Predicted index: 3 , species: Common Chickweed
Predicted index: 10 , species: Small-flowered Cranes

In [9]:
# Convert the list of species into a dataframe and export it to excel as a new column.
species_dataframe = Pandas.DataFrame(test_species_list, columns = ['species'])

# Create an excel writer object to write the output species to an excel file column.
with Pandas.ExcelWriter('Ensemble_predicted_species.xlsx') as Writer:
  species_dataframe.to_excel(Writer, sheet_name = 'Predicted Species')

print('Predicted species written to excel file.')

Predicted species written to excel file.
